# Author: Tim Harmling
- **Description:** Creates a Dataset that can be used from our Handwriting Model with *.jpg and *.txt files. The *.txt files contain the text of the handwritten text in the image. The *.jpg files contain the cropped images of the handwritten text.

# Create Dataset

In [35]:
import os
import xml.etree.ElementTree as ET
from builtins import print
from tqdm.auto import tqdm
import numpy as np

In [36]:
path = "xml"

# Get all XML file paths in path_annot and sort them
xml_files = sorted(
    [
        os.path.join(path, file_name)
        for file_name in os.listdir(path)
        if file_name.endswith(".xml")
    ]
)
 
# Get all JPEG image file paths in path_images and sort them
jpg_files = sorted(
    [
        os.path.join(path, file_name)
        for file_name in os.listdir(path)
        if file_name.endswith(".jpg")
    ]
)

In [37]:
def create_box(bbox):
    xmin = float(bbox.find("xmin").text)
    ymin = float(bbox.find("ymin").text)
    xmax = float(bbox.find("xmax").text)
    ymax = float(bbox.find("ymax").text)
    return [xmin, ymin, xmax, ymax]

In [38]:
def parse_annotation_fake(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    image_name = root.find("filename").text
    image_path = f'images/{image_name}'

    boxes = []
    classes = []
    main_classes = []
    sub_classes = []
    main_boxes = []
    sub_boxes = []
    values = []
    
    for obj in root.iter("object"):
        cls = obj.find("name").text
        classes.append(cls)
        
        bbox = obj.find("bndbox")
        boxes.append(create_box(bbox))
    
    
        attributes = obj.findall("attributes/attribute")
        for attribute in attributes:
            value_element = attribute.find("value")
            value = value_element.text
            if value is None:
                continue
            elif value is None or value.lower() in ["true", "false"]:
                continue
            
            # Versuche, den Wert in einen Float zu konvertieren
            try:
                float_value = float(value)
                int_value = int(float_value)
                values.append(str(int_value))  # Hier wird der Integer in einen String umgewandelt
            except ValueError:
                # Wenn die Konvertierung fehlschlägt, füge den originalen Wert zur Liste hinzu
                values.append(value)
            break
        
        bbox = obj.find("bndbox")
        sub_boxes.append(create_box(bbox))
        sub_classes.append(cls)
    return image_path, sub_boxes, sub_classes, values

In [39]:
from collections import namedtuple
ImageInfo = namedtuple('ImageInfo', ['path', 'boxes', 'values', 'classes'])
image_list = []
for xml_file in tqdm(xml_files): 
    image_path, boxes, classes, values = parse_annotation_fake(xml_file)
    image_list.append(ImageInfo(image_path, boxes, values, classes))
 

100%|██████████| 31/31 [00:00<00:00, 610.36it/s]



### Crop ROI

In [45]:
import cv2
def crop(xmin, ymin, xmax, ymax, image_path, crop_left_percent, crop_bottom_percent):
    image = cv2.imread(image_path)
    crop_left = int((xmax-xmin) * crop_left_percent)
    crop_bottom = int((ymax-ymin) * crop_bottom_percent)
    if image is None: 
        print("Image is None")
        return None
    cropped_image = image[int(ymin):int(ymax)-crop_bottom, int(xmin)+crop_left:int(xmax)]
    if cropped_image.size == 0:
        print("Cropped image is empty")
        return None
    return cropped_image


In [41]:
save_path_crops = "transfer_dataset"

In [42]:
cropping_params = {
    "ad_erzieher_name":         {"left": 0.1, "bottom": 0},
    "ad_erzieher_vorname":      {"left": 0.125, "bottom": 0},
    "ad_erzieher_tel":          {"left": 0.2, "bottom": 0},
    "ad_erzieher_email":        {"left": 0.1, "bottom": 0},
    "schueler_name":            {"left": 0.1, "bottom": 0},
    "schueler_vorname":         {"left": 0.15, "bottom": 0},
    "schueler_klasse":          {"left": 0.125, "bottom": 0},
    "ad_neue_ad_str_haus_nr":   {"left": 0.275, "bottom": 0},
    "ad_neue_ad_plz":           {"left": 0.175, "bottom": 0},
    "ad_neue_ad_stadt":         {"left": 0.1, "bottom": 0},
    "ad_schueler_datum":        {"left": 0.2, "bottom": 0},
    "ag_auswahl_wahl_1":        {"left": 0.15, "bottom": 0},
    "ag_auswahl_wahl_2":        {"left": 0.15, "bottom": 0},
    "ag_auswahl_wahl_3":        {"left": 0.15, "bottom": 0},
    "ag_schueler_datum":        {"left": 0.3, "bottom": 0},
    #"ad_schueler_unterschrift": {"left": 0.2, "bottom": 0.4}, 
    #"ag_schueler_unterschrift": {"left": 0.2, "bottom": 0.4}, 
}

In [46]:
for index, image in enumerate(image_list):
    boxes = image.boxes
    for i, box in enumerate(boxes):
        xmin, ymin, xmax, ymax = np.array(box)
        crop_left_percent = 0
        crop_bottom_percent = 0
        if image.classes[i] in cropping_params:
            params = cropping_params[image.classes[i]]
            crop_left_percent = float(params["left"])
            crop_bottom_percent = float(params["bottom"])
        else:
            continue

        imgCropped = crop(xmin, ymin, xmax, ymax, image.path, crop_left_percent, crop_bottom_percent)
        if imgCropped is not None:
            if image.values[i] == "0":
                continue
            img_file_path = f"{save_path_crops}/v2{index}_{i}.jpg"
            txt_file_path = f"{save_path_crops}/v2{index}_{i}.txt"
            cv2.imwrite(img_file_path, imgCropped)
            with open(txt_file_path, 'w', encoding='utf-8') as file:
                image.values[i] = image.values[i].replace(" ", "|")
                file.write(image.values[i])
                print(f"{image.classes[i]}: {image.values[i]}")


ad_erzieher_name: Böhm
ad_erzieher_vorname: Anna
ad_erzieher_tel: 5090886
ad_erzieher_email: annab@web.de
schueler_name: Böhm
schueler_vorname: Matteo
schueler_klasse: 6B
ad_neue_ad_str_haus_nr: Kurfürstenallee|23
ad_neue_ad_plz: 28211
ad_neue_ad_stadt: Bremen
ad_schueler_datum: 17/04/2024
ad_erzieher_name: Buchholz-Hering
ad_erzieher_vorname: Draheim
ad_erzieher_tel: 8733210
ad_erzieher_email: hering@gmail.com
schueler_name: Buchholz-Hering
schueler_vorname: Wojciech
schueler_klasse: 12A
ad_neue_ad_str_haus_nr: Fesenfeld|2
ad_neue_ad_plz: 28203
ad_neue_ad_stadt: Bremen
ad_schueler_datum: 18/04/2024
ad_erzieher_name: Kühn
ad_erzieher_vorname: Dirk
ad_erzieher_tel: 4885050
ad_erzieher_email: kuehners@web.de
schueler_name: Kühn
schueler_vorname: Paul
schueler_klasse: 6B
ad_neue_ad_str_haus_nr: Lilienthaler|Heerstraße|57
ad_neue_ad_plz: 28359
ad_neue_ad_stadt: Bremen
ad_schueler_datum: 20/04/2024
ad_erzieher_name: Kobelt
ad_erzieher_vorname: Lars
ad_erzieher_tel: 2976064
ad_erzieher_email

[ WARN:0@229.578] global loadsave.cpp:248 findDecoder imread_('images/AD_038.png'): can't open/read file: check file path/integrity
[ WARN:0@229.579] global loadsave.cpp:248 findDecoder imread_('images/AD_038.png'): can't open/read file: check file path/integrity
[ WARN:0@229.579] global loadsave.cpp:248 findDecoder imread_('images/AD_038.png'): can't open/read file: check file path/integrity
[ WARN:0@229.580] global loadsave.cpp:248 findDecoder imread_('images/AD_038.png'): can't open/read file: check file path/integrity
[ WARN:0@229.580] global loadsave.cpp:248 findDecoder imread_('images/AD_038.png'): can't open/read file: check file path/integrity
[ WARN:0@229.581] global loadsave.cpp:248 findDecoder imread_('images/AD_038.png'): can't open/read file: check file path/integrity
[ WARN:0@229.581] global loadsave.cpp:248 findDecoder imread_('images/AD_038.png'): can't open/read file: check file path/integrity
[ WARN:0@229.581] global loadsave.cpp:248 findDecoder imread_('images/AD_038

schueler_klasse: 6B
schueler_vorname: Sven
schueler_name: Hartmann
ad_erzieher_email: hartmann@hotmail.de
ad_erzieher_tel: 8908851
ad_erzieher_vorname: Matthias
ad_erzieher_name: Hartmann
ad_schueler_datum: 21.04.2024
ad_neue_ad_stadt: Bremen
ad_neue_ad_plz: 28211
ad_neue_ad_str_haus_nr: Großbeerenstraße|25
ad_schueler_datum: 22.04.2024
ad_erzieher_name: Pröschke
ad_erzieher_vorname: Marco
ad_erzieher_tel: 1382429
ad_erzieher_email: proeschkem@gmail.com
schueler_name: Pröschke
schueler_vorname: Krystyna
schueler_klasse: 12A
ad_neue_ad_str_haus_nr: Fesenfeld|17
ad_neue_ad_plz: 28203
ad_neue_ad_stadt: Bremen
ad_schueler_datum: 23.04.2024
ad_neue_ad_stadt: Bremen
ad_neue_ad_plz: 28207
ad_neue_ad_str_haus_nr: Osnabrücker|Straße|49
schueler_klasse: 8D
schueler_vorname: Lisa
schueler_name: Schlosser
ad_erzieher_email: bjoern-s@posteo.de
ad_erzieher_tel: 5764283
ad_erzieher_vorname: Björn
ad_erzieher_name: Schlosser
schueler_vorname: Constance
ad_neue_ad_plz: 28201
ad_neue_ad_stadt: Bremen
ad

[ WARN:0@236.214] global loadsave.cpp:248 findDecoder imread_('images/AG_046.png'): can't open/read file: check file path/integrity
[ WARN:0@236.216] global loadsave.cpp:248 findDecoder imread_('images/AG_046.png'): can't open/read file: check file path/integrity
[ WARN:0@236.216] global loadsave.cpp:248 findDecoder imread_('images/AG_046.png'): can't open/read file: check file path/integrity
[ WARN:0@236.217] global loadsave.cpp:248 findDecoder imread_('images/AG_046.png'): can't open/read file: check file path/integrity
[ WARN:0@236.217] global loadsave.cpp:248 findDecoder imread_('images/AG_046.png'): can't open/read file: check file path/integrity
[ WARN:0@236.217] global loadsave.cpp:248 findDecoder imread_('images/AG_046.png'): can't open/read file: check file path/integrity
[ WARN:0@236.218] global loadsave.cpp:248 findDecoder imread_('images/AG_046.png'): can't open/read file: check file path/integrity
[ WARN:0@236.218] global loadsave.cpp:248 findDecoder imread_('images/AG_047

ag_auswahl_wahl_1: Fußball
ag_auswahl_wahl_2: Robotik
ag_auswahl_wahl_3: Programmierung
ag_schueler_datum: 13.04.2024
schueler_name: Ahmad
schueler_vorname: Muhamed
schueler_klasse: 8b
ag_auswahl_wahl_1: Fußball
ag_auswahl_wahl_2: Robotik
ag_auswahl_wahl_3: Programmierung
ag_schueler_datum: 13.04.2024
schueler_name: Laue
schueler_vorname: Max
schueler_klasse: 6C
ag_auswahl_wahl_1: Basketball
ag_auswahl_wahl_2: Volleyball
ag_auswahl_wahl_3: Chor
ag_schueler_datum: 11.04.2024
schueler_name: Laue
schueler_vorname: Max
schueler_klasse: 6C
ag_auswahl_wahl_1: Basketball
ag_auswahl_wahl_2: Volleyball
ag_auswahl_wahl_3: Chor
ag_schueler_datum: 11.04.2024
schueler_name: Ahmad
schueler_vorname: Muhamad
schueler_klasse: 8b
ag_auswahl_wahl_1: Fußball
ag_auswahl_wahl_2: Robotik
ag_auswahl_wahl_3: Programmierung
ag_schueler_datum: 13.04.2024
schueler_name: Laue
schueler_vorname: Max
schueler_klasse: 6C
ag_auswahl_wahl_1: Basketball
ag_auswahl_wahl_2: Volleyball
ag_auswahl_wahl_3: Chor
ag_schueler_da